In [1]:
%config IPCompleter.greedy=True
import pandas as pd
import numpy as np

In [2]:
kickstarter_projects_2018 = pd.read_csv('ks-projects-201801.csv')
print(kickstarter_projects_2018.shape)
kickstarter_projects_2018.head()

(378661, 15)


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [3]:
kickstarter_projects_2018.columns

Index(['ID', 'name', 'category', 'main_category', 'currency', 'deadline',
       'goal', 'launched', 'pledged', 'state', 'backers', 'country',
       'usd pledged', 'usd_pledged_real', 'usd_goal_real'],
      dtype='object')

In [5]:
print(kickstarter_projects_2018.isnull().any().any())
print(kickstarter_projects_2018.isnull().any().sum())
print(kickstarter_projects_2018.isnull().sum())

True
2
ID                     0
name                   4
category               0
main_category          0
currency               0
deadline               0
goal                   0
launched               0
pledged                0
state                  0
backers                0
country                0
usd pledged         3797
usd_pledged_real       0
usd_goal_real          0
dtype: int64


In [6]:
kickstarter_projects_2018 ['country'].value_counts().count()

23

In [7]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [8]:
basket = (kickstarter_projects_2018[kickstarter_projects_2018 ['country'] =="US"]
          .groupby(['main_category', 'state'])['category']
          .sum().unstack().reset_index().fillna(0)
          .set_index('main_category'))

In [9]:
basket.head()

state,canceled,failed,live,successful,suspended
main_category,,,,,
Art,Mixed MediaPaintingArtDigital ArtDigital ArtPe...,Public ArtIllustrationPublic ArtArtArtPublic A...,IllustrationPerformance ArtArtPerformance ArtA...,ArtPaintingIllustrationIllustrationCeramicsPub...,ArtIllustrationInstallationsSculpturePerforman...
Comics,WebcomicsComic BooksComicsComic BooksComicsCom...,Comic BooksComicsComicsComicsComicsComicsComic...,ComicsWebcomicsComicsComic BooksComic BooksWeb...,ComicsComicsComicsComic BooksComic BooksComic ...,EventsComicsComicsComicsGraphic NovelsGraphic ...
Crafts,DIYDIYCraftsCraftsCraftsCrochetCandlesCraftsQu...,CraftsCraftsDIYCraftsDIYWoodworkingWoodworking...,DIYStationeryWoodworkingCraftsDIYWoodworkingCr...,KnittingCraftsCraftsCraftsCraftsDIYCraftsDIYLe...,CraftsCraftsCraftsDIYDIYWeavingCraftsCraftsDIY...
Dance,DanceSpacesDanceDanceDancePerformancesDanceSpa...,DancePerformancesPerformancesSpacesPerformance...,PerformancesDancePerformancesPerformancesSpace...,DancePerformancesDanceDanceDanceDanceDanceDanc...,DanceWorkshopsPerformancesDanceWorkshopsSpaces...
Design,Product DesignProduct DesignArchitectureProduc...,DesignProduct DesignProduct DesignProduct Desi...,Product DesignProduct DesignProduct DesignProd...,Product DesignProduct DesignProduct DesignDesi...,Product DesignArchitectureProduct DesignProduc...


In [11]:
basket.iloc[:,[0,1,2]].head()

state,canceled,failed,live
main_category,,,
Art,Mixed MediaPaintingArtDigital ArtDigital ArtPe...,Public ArtIllustrationPublic ArtArtArtPublic A...,IllustrationPerformance ArtArtPerformance ArtA...
Comics,WebcomicsComic BooksComicsComic BooksComicsCom...,Comic BooksComicsComicsComicsComicsComicsComic...,ComicsWebcomicsComicsComic BooksComic BooksWeb...
Crafts,DIYDIYCraftsCraftsCraftsCrochetCandlesCraftsQu...,CraftsCraftsDIYCraftsDIYWoodworkingWoodworking...,DIYStationeryWoodworkingCraftsDIYWoodworkingCr...
Dance,DanceSpacesDanceDanceDancePerformancesDanceSpa...,DancePerformancesPerformancesSpacesPerformance...,PerformancesDancePerformancesPerformancesSpace...
Design,Product DesignProduct DesignArchitectureProduc...,DesignProduct DesignProduct DesignProduct Desi...,Product DesignProduct DesignProduct DesignProd...


In [12]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1